In [2]:
import numpy as np
import os 

In [3]:
dirname = './02-04/'
a0 = -45
a1 = 0 
b0 = -22.5 
b1 = 22.5

angles = [
    (a0, b0),
    (a0 + 90, b0 + 90),
    (a0 + 90, b0),
    (a0, b0 + 90),
    (a0, b1),
    (a0 + 90, b1 + 90),
    (a0 + 90, b1),
    (a0, b1 + 90),
    (a1, b0),
    (a1 + 90, b0 + 90),
    (a1 + 90, b0),
    (a1, b0 + 90),
    (a1, b1),
    (a1 + 90, b1 + 90),
    (a1 + 90, b1),
    (a1, b1 + 90),
]

coinc_window = 40e-9
acq_time = 1

In [ ]:
n = {} 
for ang in angles: 
    n[ang] = np.mean(np.loadtxt(os.path.join(dirname, f'{ang[0]}_{ang[1]}.txt')), axis=0)[-1]

nb = {} 
for ang in angles: 
    nb[ang] = np.mean(np.loadtxt(os.path.join(dirname, f'{ang[0]}_{ang[1]}.txt')), axis=0)[1]

nbp = {} 
for ang in angles: 
    nbp[ang] = np.mean(np.loadtxt(os.path.join(dirname, f'{ang[0]}_{ang[1]}.txt')), axis=0)[1]


{(-45, -22.5): np.float64(25973.016666666666),
 (45, 67.5): np.float64(26478.433333333334),
 (45, -22.5): np.float64(25584.7),
 (-45, 67.5): np.float64(26183.716666666667),
 (-45, 22.5): np.float64(26187.733333333334),
 (45, 112.5): np.float64(25490.416666666668),
 (45, 22.5): np.float64(26013.483333333334),
 (-45, 112.5): np.float64(26440.1),
 (0, -22.5): np.float64(24331.85),
 (90, 67.5): np.float64(28611.0),
 (90, -22.5): np.float64(28959.45),
 (0, 67.5): np.float64(24228.95),
 (0, 22.5): np.float64(24371.433333333334),
 (90, 112.5): np.float64(27877.116666666665),
 (90, 22.5): np.float64(28550.5),
 (0, 112.5): np.float64(24189.733333333334)}

In [ ]:
# a0 b0
n_a0_b0   = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b0}.txt')), axis=0)[-1]
n_a0p_b0p = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b0 + 90}.txt')), axis=0)[-1]
n_a0p_b0  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b0}.txt')), axis=0)[-1]
n_a0_b0p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b0 + 90}.txt')), axis=0)[-1]

nb_a0_b0   = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b0}.txt')), axis=0)[1]
nb_a0p_b0p = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b0 + 90}.txt')), axis=0)[1]
nb_a0p_b0  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b0}.txt')), axis=0)[1]
nb_a0_b0p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b0 + 90}.txt')), axis=0)[1]

nbp_a0_b0   = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b0}.txt')), axis=0)[3]
nbp_a0p_b0p = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b0 + 90}.txt')), axis=0)[3]
nbp_a0p_b0  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b0}.txt')), axis=0)[3]
nbp_a0_b0p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b0 + 90}.txt')), axis=0)[3]

# a0 b1
n_a0_b1   = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b1}.txt')), axis=0)[-1]
n_a0p_b1p = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b1 + 90}.txt')), axis=0)[-1]
n_a0p_b1  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b1}.txt')), axis=0)[-1]
n_a0_b1p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b1 + 90}.txt')), axis=0)[-1]

nb_a0_b1   = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b1}.txt')), axis=0)[1]
nb_a0p_b1p = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b1 + 90}.txt')), axis=0)[1]
nb_a0p_b1  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b1}.txt')), axis=0)[1]
nb_a0_b1p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b1 + 90}.txt')), axis=0)[1]

nbp_a0_b1   = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b1}.txt')), axis=0)[3]
nbp_a0p_b1p = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b1 + 90}.txt')), axis=0)[3]
nbp_a0p_b1  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0 + 90}_{b1}.txt')), axis=0)[3]
nbp_a0_b1p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a0}_{b1 + 90}.txt')), axis=0)[3]

# a1 b0
n_a1_b0   = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b0}.txt')), axis=0)[-1]
n_a1p_b0p = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b0 + 90}.txt')), axis=0)[-1]
n_a1p_b0  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b0}.txt')), axis=0)[-1]
n_a1_b0p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b0 + 90}.txt')), axis=0)[-1]

nb_a1_b0   = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b0}.txt')), axis=0)[1]
nb_a1p_b0p = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b0 + 90}.txt')), axis=0)[1]
nb_a1p_b0  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b0}.txt')), axis=0)[1]
nb_a1_b0p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b0 + 90}.txt')), axis=0)[1]

nbp_a1_b0   = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b0}.txt')), axis=0)[3]
nbp_a1p_b0p = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b0 + 90}.txt')), axis=0)[3]
nbp_a1p_b0  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b0}.txt')), axis=0)[3]
nbp_a1_b0p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b0 + 90}.txt')), axis=0)[3]

# a1 b1
n_a1_b1   = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b1}.txt')), axis=0)[-1]
n_a1p_b1p = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b1 + 90}.txt')), axis=0)[-1]
n_a1p_b1  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b1}.txt')), axis=0)[-1]
n_a1_b1p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b1 + 90}.txt')), axis=0)[-1]

nb_a1_b1   = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b1}.txt')), axis=0)[1]
nb_a1p_b1p = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b1 + 90}.txt')), axis=0)[1]
nb_a1p_b1  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b1}.txt')), axis=0)[1]
nb_a1_b1p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b1 + 90}.txt')), axis=0)[1]

nbp_a1_b1   = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b1}.txt')), axis=0)[3]
nbp_a1p_b1p = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b1 + 90}.txt')), axis=0)[3]
nbp_a1p_b1  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1 + 90}_{b1}.txt')), axis=0)[3]
nbp_a1_b1p  = np.mean(np.loadtxt(os.path.join(dirname, f'{a1}_{b1 + 90}.txt')), axis=0)[3]

## Calculate S

In [13]:
def E(alpha, beta): 
    a_b, ap_b, a_bp, ap_bp = [n[(alpha, beta)], n[(alpha + 90, beta)], n[(alpha, beta + 90)], n[(alpha + 90, beta + 90)]]
    return (a_b + ap_bp - a_bp - ap_b) / (a_b + ap_bp + a_bp + ap_b)

S = E(a0, b0) - E(a0, b1) + E(a1, b0) + E(a1, b1)

## Error propagation

In [ ]:
ps = [(0, 0), (90, 90), (90, 0), (0, 90)]
dss = {}
for a, b in [(a0, b0), (a1, b0), (a0, b1), (a1, b1)]:
    ds = 0
    ntot = 0 

    for pa, pb in ps:
        ntot += n[(a + pa, b + pb)]

    for pa, pb, in ps: 
        factor = 1
        if pa + pb == 90: 
            factor = -1
        ds += factor * n[(a + pa, b + pb)]

    for pa, pb in ps: 
        if pa + pb == 90: 
            ds += -ntot
        else: 
            ds += ntot 
    
        dss[(a + pa, b + pb)] = ds / ntot**2 


ntot_a0_b0 = n[(a0, b0)] + n[(a0 + 90, b0 + 90)] + n[(a0 + 90, b0)] + n[(a0, b0 + 90)]
ntot_a1_b0 = n[(a1, b0)] + n[(a1 + 90, b0 + 90)] + n[(a1 + 90, b0)] + n[(a1, b0 + 90)]
ntot_a0_b1 = n[(a0, b1)] + n[(a0 + 90, b1 + 90)] + n[(a0 + 90, b1)] + n[(a0, b1 + 90)]
ntot_a1_b1 = n[(a1, b1)] + n[(a1 + 90, b1 + 90)] + n[(a1 + 90, b1)] + n[(a1, b1 + 90)]

# s_a0_b0   = (ntot_a0_b0 - n_a0_b0 - n_a0p_b0p + n_a0p_b0 + n_a0_b0p) / ntot_a0_b0**2
# s_a0p_b0p = (ntot_a0_b0 - n_a0_b0 - n_a0p_b0p + n_a0p_b0 + n_a0_b0p) / ntot_a0_b0**2
# s_a0p_b0  = (-ntot_a0_b0 - n_a0_b0 - n_a0p_b0p + n_a0p_b0 + n_a0_b0p) / ntot_a0_b0**2
# s_a0_b0p  = (-ntot_a0_b0 - n_a0_b0 - n_a0p_b0p + n_a0p_b0 + n_a0_b0p) / ntot_a0_b0**2

# s_a1_b0   = (ntot_a1_b0 - n_a1_b0 - n_a1p_b0p + n_a1p_b0 + n_a1_b0p) / ntot_a1_b0**2
# s_a1p_b0p = (ntot_a1_b0 - n_a1_b0 - n_a1p_b0p + n_a1p_b0 + n_a1_b0p) / ntot_a1_b0**2
# s_a1p_b0  = (-ntot_a1_b0 - n_a1_b0 - n_a1p_b0p + n_a1p_b0 + n_a1_b0p) / ntot_a1_b0**2
# s_a1_b0p  = (-ntot_a1_b0 - n_a1_b0 - n_a1p_b0p + n_a1p_b0 + n_a1_b0p) / ntot_a1_b0**2

# s_a0_b1   = (ntot_a0_b1 - n_a0_b1 - n_a0p_b1p + n_a0p_b1 + n_a0_b1p) / ntot_a0_b1**2
# s_a0p_b1p = (ntot_a0_b1 - n_a0_b1 - n_a0p_b1p + n_a0p_b1 + n_a0_b1p) / ntot_a0_b1**2
# s_a0p_b1  = (-ntot_a0_b1 - n_a0_b1 - n_a0p_b1p + n_a0p_b1 + n_a0_b1p) / ntot_a0_b1**2
# s_a0_b1p  = (-ntot_a0_b1 - n_a0_b1 - n_a0p_b1p + n_a0p_b1 + n_a0_b1p) / ntot_a0_b1**2

# s_a1_b1   = (ntot_a1_b1 - n_a1_b1 - n_a1p_b1p + n_a1p_b1 + n_a1_b1p) / ntot_a1_b1**2
# s_a1p_b1p = (ntot_a1_b1 - n_a1_b1 - n_a1p_b1p + n_a1p_b1 + n_a1_b1p) / ntot_a1_b1**2
# s_a1p_b1  = (-ntot_a1_b1 - n_a1_b1 - n_a1p_b1p + n_a1p_b1 + n_a1_b1p) / ntot_a1_b1**2
# s_a1_b1p  = (-ntot_a1_b1 - n_a1_b1 - n_a1p_b1p + n_a1p_b1 + n_a1_b1p) / ntot_a1_b1**2

# ns = [
#         n_a0_b0, n_a0p_b0p, n_a0p_b0, n_a0_b0p,
#         n_a0_b1, n_a0p_b1p, n_a0p_b1, n_a0_b1p,
#         n_a1_b0, n_a1p_b0p, n_a1p_b0, n_a1_b0p,
#         n_a1_b1, n_a1p_b1p, n_a1p_b1, n_a1_b1p
#       ]
# s_derivs = [
#         s_a0_b0, s_a0p_b0p, s_a0p_b0, s_a0_b0p,
#         s_a0_b1, s_a0p_b1p, s_a0p_b1, s_a0_b1p,
#         s_a1_b0, s_a1p_b0p, s_a1p_b0, s_a1_b0p,
#         s_a1_b1, s_a1p_b1p, s_a1p_b1, s_a1_b1p
#       ]


std = np.sqrt(sum(n[(a, b)] * dss[(a, b)]**2 for a, b in angles))
print(f'S = {'{:.4}'.format(S)} ± {'{:.3}'.format(std)}') 

S = 2.378 ± 0.0822
